In [6]:
import pandas as pd
import ast

# Read the csv file
df= pd.read_csv('data/labels.csv')

# Replace the personal path to data/
df['image_path'] = df['image_path'].str.replace('^/media/tom/Elements/rgb_dataset_multi_patches', 'data', regex=True)

# Drop the 'original' column, we don't need it
df.drop('original', axis=1, inplace=True)

# Drop the 'Unnamed: 0' column, we don't need it
df.drop('Unnamed: 0', axis=1, inplace=True)

# Delete the rows from the empty folder, because they don't have a box nor class
df = df[~df['image_path'].str.contains('empty')]

# Convert string representation to actual list of dictionaries
df['objects'] = df['objects'].apply(ast.literal_eval)

# Extract 'box' and 'class' into new columns
df['boxes'] = df['objects'].apply(lambda x: [d['box'] for d in x])
df['classes'] = df['objects'].apply(lambda x: [d['class'] for d in x])

# Drop the 'objects' column if it's no longer necessary
df.drop('objects', axis=1, inplace=True)

# Check bouding boxes validity
df['is_valid'] = True

for i, row in df.iterrows():
    boxes = row['boxes']
    for box in boxes:
        x_min, y_min, x_max, y_max = box

         # Calculate width and height
        width = x_max - x_min
        height = y_max - y_min

        # Check if both width and height are positive
        if width <= 0 or height <= 0:
            df.at[i, 'is_valid'] = False

# Delete the rows from the empty folder, because they don't have a box nor class
df = df[df['is_valid']]

# Write the file
df.to_csv('data/labels2.csv', index=False)